In [1]:
!pip install pyarrow fastparquet huggingface_hub evaluate rouge_score langchain-openai transformers langchain langchainhub langchain-huggingface faiss-cpu langchain-community sentence_transformers

In [2]:
import pandas as pd
import evaluate
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
import os
import getpass
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.llms import HuggingFaceHub

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ucsc_passages_df = pd.read_csv("/root/passage.csv", index_col=0)
ucsc_passages_df

,passage,link,comments
Index,,,
0,"Our club exists to give community members, UC ...",https://recreation.ucsc.edu/boating/community-...,NaN
1,Boating Club Hours\nThe Community Boating Cent...,https://recreation.ucsc.edu/boating/index.html,NaN
2,Our boats:\nSailing vessels for weekend use cu...,https://recreation.ucsc.edu/boating/community-...,NaN
3,Club Membership Fees:\n\nUCSC Student Membersh...,https://recreation.ucsc.edu/boating/community-...,NaN
4,Summer Boating Classes\nEnrollment for summer ...,https://recreation.ucsc.edu/boating/index.html,NaN
...,...,...,...
59,Transfer Admission Guarantee\nTransfer Admissi...,https://admissions.ucsc.edu/transfer-admission...,NaN
60,UC Santa Cruz\nWith its jaw-dropping coastline...,https://admission.universityofcalifornia.edu/c...,NaN
61,Santa Cruz: First-year admit data\nThis is a s...,https://admission.universityofcalifornia.edu/c...,NaN


In [4]:
ucsc_qa_df = pd.read_csv("/root/new_qa.csv", index_col=0)
ucsc_qa_df

,questions,answers,source_name,source,generated_response
index,,,,,
0,Who can be a member of the Boating Club?,Everyone is welcome to join! If you do not hav...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,Members of the Boating Club can include commun...
1,What classes does the boating center offer to ...,We offer all levels of sailing (dinghy and kee...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating center offers classes for beginner...
2,When is the boating club open?,"The club is open year-round, Saturday and Sund...",UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating club is open on Saturday and Sunda...
3,What boats are available to members?,Sailing vessels for weekend use currently incl...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,Members have access to the following boats:\n\...
4,How much does a boating membership cost?,UCSC Student Membership: $45\nNon-Student Quar...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The provided context does not include any info...
...,...,...,...,...,...
107,What if I don't fulfill the general education ...,If you don't satisfy general education require...,Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,If you do not complete the IGETC or partial IG...
108,What is UC TAP?,UC Transfer Admission Planner (UC TAP) is an o...,Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,"UC TAP, or the UC Transfer Admission Planner, ..."
109,When are acceptance notices sent out?,"For fall quarter acceptance, notices are sent ...",Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,The context provided does not specify when acc...


In [5]:
open_ai_llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key="sk-proj-yKHe10Y7zEhFX1sncOFCT3BlbkFJh31ATScEQmYjQRlNpiLg")
open_ai_llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f43dcfd3730>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f43dce5d060>, root_client=<openai.OpenAI object at 0x7f455dbbc190>, root_async_client=<openai.AsyncOpenAI object at 0x7f43dcfd3760>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')

In [6]:
ucsc_passge_data_loader = DataFrameLoader(ucsc_passages_df, page_content_column="passage")
ucsc_passage_data = ucsc_passge_data_loader.load()
ucsc_passage_data[0]

Document(metadata={'link': 'https://recreation.ucsc.edu/boating/community-club.html', ' comments': nan}, page_content='Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.')

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = text_splitter.split_documents(ucsc_passage_data)
len(docs)

Created a chunk of size 324, which is longer than the specified 200
Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 451, which is longer than the specified 200
Created a chunk of size 245, which is longer than the specified 200
Created a chunk of size 226, which is longer than the specified 200
Created a chunk of size 580, which is longer than the specified 200
Created a chunk of size 1970, which is longer than the specified 200
Created a chunk of size 312, which is longer than the specified 200
Created a chunk of size 304, which is longer than the specified 200
Created a chunk of size 389, which is longer than the specified 200
Created a chunk of size 697, which is longer than the specified 200
Created a chunk of size 841, which is longer than the specified 200
Created a chunk of size 501, which is longer than the specified 200
Created a chunk of size 743, which is longer than the specified 200
Created a chunk of size 787, which is longer th

455

In [8]:
embeddings = HuggingFaceEmbeddings()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
db = FAISS.from_documents(docs, embeddings)

In [10]:
index = 1
question = ucsc_qa_df["questions"][index]
question

'What classes does the boating center offer to community members?'

In [11]:
display(ucsc_qa_df["answers"][index])

'We offer all levels of sailing (dinghy and keelboats) and rowing classes to community members during the summer. Registration for Summer boating classes opens on March 15th. For questions about course offerings or registration, please call the following number: (831) 425-1164 or (949) 371-1448.'

In [12]:
query = question
docs = db.similarity_search(query)
docs

[Document(metadata={'link': 'https://recreation.ucsc.edu/boating/index.html', ' comments': nan}, page_content="UC Santa Cruz Community Boating Center is a community-based boating program that connects students and community members to the Monterey Bay through recreational activities including sailing, rowing and kayaking.\nUC Santa Cruz Community Boating Center staff and veteran participants provide a familiarity with the equipment, skills, and water safety that comes from a lifetime of enjoying these sports.\nThe Community Boating Center also connects people who are interested in these sports to one another, whether you are a beginner seeking to learn something new or a experienced sailor, rower, and/or kayaker looking to improve your skills and get time on the water. Whether you've never been on a boat before, or want to build on old skills, we have a class for you! From 2-week junior summer camps to quarterly physical education classes for UC Santa Cruz Students, you'll be cruising 

In [13]:
ucsc_passages_df.drop_duplicates(subset=["passage"], inplace=True)
ucsc_passages_df

,passage,link,comments
Index,,,
0,"Our club exists to give community members, UC ...",https://recreation.ucsc.edu/boating/community-...,NaN
1,Boating Club Hours\nThe Community Boating Cent...,https://recreation.ucsc.edu/boating/index.html,NaN
2,Our boats:\nSailing vessels for weekend use cu...,https://recreation.ucsc.edu/boating/community-...,NaN
3,Club Membership Fees:\n\nUCSC Student Membersh...,https://recreation.ucsc.edu/boating/community-...,NaN
4,Summer Boating Classes\nEnrollment for summer ...,https://recreation.ucsc.edu/boating/index.html,NaN
...,...,...,...
59,Transfer Admission Guarantee\nTransfer Admissi...,https://admissions.ucsc.edu/transfer-admission...,NaN
60,UC Santa Cruz\nWith its jaw-dropping coastline...,https://admission.universityofcalifornia.edu/c...,NaN
61,Santa Cruz: First-year admit data\nThis is a s...,https://admission.universityofcalifornia.edu/c...,NaN


In [24]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = db.as_retriever()

template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    x = "\n\n".join(doc.page_content for doc in docs)
    print(x)
    return x

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | open_ai_llm
    | StrOutputParser()
)

llm_answer = rag_chain.invoke(question)


UC Santa Cruz Community Boating Center is a community-based boating program that connects students and community members to the Monterey Bay through recreational activities including sailing, rowing and kayaking.
UC Santa Cruz Community Boating Center staff and veteran participants provide a familiarity with the equipment, skills, and water safety that comes from a lifetime of enjoying these sports.
The Community Boating Center also connects people who are interested in these sports to one another, whether you are a beginner seeking to learn something new or a experienced sailor, rower, and/or kayaker looking to improve your skills and get time on the water. Whether you've never been on a boat before, or want to build on old skills, we have a class for you! From 2-week junior summer camps to quarterly physical education classes for UC Santa Cruz Students, you'll be cruising around Monterey Bay with ease.

Our club exists to give community members, UC Santa Cruz students, faculty, and s

In [25]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()
retriever = db.as_retriever()

source_docs = []
rag_answers = []

def format_docs(docs):
    ret_docs = "\n\n".join(doc.page_content for doc in docs)
    source_docs.append(ret_docs)
    return ret_docs

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | open_ai_llm
    | output_parser
)

for q in ucsc_qa_df['questions']:
  answer = rag_chain.invoke(q)
  rag_answers.append(answer)

In [26]:
ucsc_qa_df["generated_response"] = rag_answers

In [27]:
ucsc_qa_df["source_docs"] = source_docs

In [28]:
ucsc_qa_df

,questions,answers,source_name,source,generated_response,source_docs
index,,,,,,
0,Who can be a member of the Boating Club?,Everyone is welcome to join! If you do not hav...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,Members of the Boating Club can include commun...,"Our club exists to give community members, UC ..."
1,What classes does the boating center offer to ...,We offer all levels of sailing (dinghy and kee...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating center offers classes for beginner...,UC Santa Cruz Community Boating Center is a co...
2,When is the boating club open?,"The club is open year-round, Saturday and Sund...",UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating club is open on Saturday and Sunda...,Boating Club Hours\nThe Community Boating Cent...
3,What boats are available to members?,Sailing vessels for weekend use currently incl...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,Members have access to the following sailing v...,Our boats:\nSailing vessels for weekend use cu...
4,How much does a boating membership cost?,UCSC Student Membership: $45\nNon-Student Quar...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The provided context does not include any info...,There are a few minor fees associated with tak...
...,...,...,...,...,...,...
107,What if I don't fulfill the general education ...,If you don't satisfy general education require...,Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,If you do not complete the IGETC or partial IG...,"By the end of the fall term prior to transfer,..."
108,What is UC TAP?,UC Transfer Admission Planner (UC TAP) is an o...,Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,"UC TAP, or the UC Transfer Admission Planner, ...",UCSC TPP\nUCSC TAG Step-by-Step\nComplete the ...
109,When are acceptance notices sent out?,"For fall quarter acceptance, notices are sent ...",Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,The context does not specify when acceptance n...,First Year\t\nApplication Submission\n\nMay 20...


In [29]:
ucsc_qa_df.to_csv("/root/new_qa.csv")

In [23]:
src_ans_list = ucsc_qa_df['answers'].tolist()

In [22]:
gen_ans_list = ucsc_qa_df["generated_response"].tolist()

In [18]:
bleu = evaluate.load("bleu")

In [19]:
bleu_results = bleu.compute(predictions=gen_ans_list, references=src_ans_list)
bleu_results

{'bleu': 0.0633104992990607,
 'precisions': [0.2622744439781788,
  0.07687935199658946,
  0.03667870036101083,
  0.021723176280639953],
 'brevity_penalty': 1.0,
 'length_ratio': 1.158858810179932,
 'translation_length': 7149,
 'reference_length': 6169}

In [20]:
rouge = evaluate.load("rouge")

In [21]:
rouge_results = rouge.compute(predictions=gen_ans_list, references=src_ans_list)
rouge_results

{'rouge1': 0.2797127786218908,
 'rouge2': 0.09247232016908774,
 'rougeL': 0.19052847117634536,
 'rougeLsum': 0.19564972576604922}